<a href="https://colab.research.google.com/github/ItoMasaki/Notebook-List/blob/main/GraspWithYOLOv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip3 install pytorch_lightning
import torch
import pytorch_lightning

from glob import glob

In [8]:
def atoi(text):    
    return int(text) if text.isdigit() else text    
    
def natural_keys(text):    
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [10]:
class LoadData():
  def __init__(self):
    T_annotations_path = "/content/drive/MyDrive/datasets/image_based_grasp_point/Annotations/*"
    T_images_path = "/content/drive/MyDrive/datasets/image_based_grasp_point/Images/*"
    
    T_annotations_pathes = sorted(glob(T_annotations_path), key=natural_keys)
    T_images_pathes = sorted(glob(T_images_path), key=natural_keys)

  def __getitem__(self, index):
    pass

  def __len__(self):
    pass